# Imports

In [ ]:
import urllib

This code is used to connect to the desired S3 buckets.

I have cleaned the file of sensitive information saved to variables, this includes:

- `AWS_S3_BUCKET`, where we would have `AWS_S3_BUCKET = '<S3_path_to_pin_bucket>'`

- `MOUNT_NAME`, where we would have `MOUNT_NAME = '<desired_mount_path>'`

Below are two functions, one to read the credentials and another to mount the s3 bucket to databricks.

In [ ]:
def read_creds(file_path="/FileStore/tables/authentication_credentials.csv"):
    ''' This function reads the AWS credentials stored on databricks.

    Args:
        file_path (`string`): The path to the CSV file containing credentials.
    
    Returns:
        cred_dict (`dictionary`): A dictionary with the credentials needed to connect to the S3 bucket.
    '''

    # Specify file type to be csv
    file_type = "csv"
    # Indicates file has first row as the header
    first_row_is_header = "true"
    # Indicates file has comma as the delimeter
    delimiter = ","
    # Read the CSV file to spark dataframe
    aws_keys_df = spark.read.format(file_type)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .load(file_path)

    SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']

    cred_dict = {
        'ACCESS_KEY' : aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID'],
        'ENCODED_SECRET_KEY' : urllib.parse.quote(string=SECRET_KEY, safe="")
    }

    return cred_dict

def mount_bucket(AWS_S3_BUCKET, MOUNT_NAME):
    ''' Mounts an S3 bucket using the provided AWS credentials and mount name.

    Args:
        AWS_S3_BUCKET (str): The name of the S3 bucket.
        MOUNT_NAME (str): The name to use for the mount point.

    Raises:
        MountException: If there is an error during the mounting process.
    '''

    cred_dict = read_creds()

    # Source url
    SOURCE_URL = "s3n://{0}:{1}@{2}".format(cred_dict['ACCESS_KEY'], cred_dict['ENCODED_SECRET_KEY'], AWS_S3_BUCKET)

    try:
        # Mount the drive
        dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
        print(f'S3 bucket {AWS_S3_BUCKET} successfully mounted at {MOUNT_NAME}.')

    except Exception as e:
        print(f'Error mounting S3 bucket {AWS_S3_BUCKET}. Error given: {e}')

For example, to mount the S3 bucket called `'test_bucket'` with mount name `'test_mount'`, one will use:

```
mount_bucket('test_bucket', 'test_mount')
```